# **Clustering**
Also know as *unsupervised classification*.

* **Given**: a set of $N$, each described by $D$ values (dimension);
* **Task**: find a *natural partitioning* in $K$ cluster and possibly a number of *noise objects*;
* **Result**: a *clustering scheme* (i.e. function mapping each data object to a sequence $[1,\dots,K]$).

Note that:
* Objects in the same cluster are similar $\to$ Maximize intra-cluster similarity;
* Objects in different clusters are not similar $\to$ Minimize inter-cluster similarity.

Type of methods:
* Partitionig (i.e. $K$-means, expectation maximization, CLARANS);
* Hierarchic (i.e. agglomerative/divisive, BIRCH, CURE);
* Based on linkage;
* Based on density (i.e DBSCAN, DENCLUE);
* Statistics (i.e. IBM-IM, COBWEB, Autoclass).

## $K$-means
Let's have some data that could be modeled as a five component gaussian mixture, how do we guess the number of cluster in a $D$-dimensional space (with $D \ge 2$).

Let's pretend we have to transmit this information, allowing only two bits per point (lossy transmission), then we'll need a encoding/decoding mechanism.

**How much is this loss?** 
* First idea: let's try with the squared errors between reals point and their encoding/decoding, considering a partition of the space into a grid of cells (decode each pairs of bit with the center of the grid cell, midpoint);
* An improvement: instead of the midpoint use the centroid of the points in the grid cell (vectorial summation divided by the number of points);
* $K$-means: ask the user the number of clusters $K$ (a bit like cheating) with a random choice of $K$ points as temporary centers. Each point finds his nearest center and for each center finds the centroid of its points (compute the centroid) and move there the center (**iteratively**). $K$-means ends with a stable solution.

Some questions:
* What are we trying to optimize? Distortion.
* Is termination guaranteed? Nope, sometimes the algorithm can loop.
* The best clustering scheme is find? What is the definition of best clustering scheme? Depends on the application.
* How should we start? Randomly.
* How can we find the number of clusters? Try and improvement.


### Distortion
Given:
* A encoding function: $\text{Encode}:\mathbb{R}^D\to[1,\dots,K]$;
* A decoding function: $\text{Decode}:[1,\dots,K]\to\mathbb{R}^D$;
* Define: $\text{Distortion}=\sum_{i=1}^N(e_i-\text{Decode}(\text{Encode}(e_i)))^2$;
* A shortcut: $\text{Decode}(k)=c_k$ (code of $k$, centers).

Then we can define: $$\text{Distortion}=\sum_{i=1}^N(e_i-c_{\text{Encode}(e_i)})^2$$

To have minimal distortion $e_i$ must:
1. Be encoded with the nearest center (otherwise the distortion could be reduced by substituting $\text{Encode}(e_i)$ with the nearest center): $$c_{\text{Encode}(e_i)}=\underset{c_j\in\{c_1,\dots,c_k\}}{\operatorname{argmin}}(e_i-c_j)^2$$
1. The partial derivative of distortion w.r.t. the position of each center must be zero (minimization $\nabla f=0$). When distortion is minimal: $$c_j=\frac{1}{|\text{OwnedBy}(c_j)|}\sum_{i \in \text{OwnedBy}(c_j)}(e_i-c_j)$$ 
3. Each center must be the centroid of the points it owns.

Algorithm: for improving a sub-optimal solution alternately steps 1 and 3. After a finite number of steps the system reaches a state where neither of the two operations changes the state (the distortion function is strictly convex).

### Algorithm termination
There's only a finite number of ways to partition $N$ objects into $K$ gropus.
The state of the algorithm is given by the two coding functions.

The number of configurations where all the centers are centroids of the points they own is finite.

If after an iteration the state changes, the distortion is reduced (since it's a convex function). 
Each change of state bring to a state which was never visited before.
Sooner or later the algorithm will stop because there are no new states reachable.

The ending state is not necessarily the best possible.

The starting point is important, usually: choose randomly the first starting point and then choose in sequence the $[2,\dots,K]$ as far as possible from the preceding ones.

Note: usually starting with different random states is enough to obtain a very good solution.

### Choosing the number of clusters
There are different methods:
* Try various values ($k$ is an hyperparameter);
* Use quantitative evaluation of the quality of the clustering sccheme to decide among the different values;
* The best value is an optimal compromise between the minimization of the intra-cluster distances and the maximization of the inter-cluster distances (need to measure them, not so trivial).


### Proximity function
The choice of the function is completely independent from the algorithm.

**Distortion (or inertia) a.k.a. sum of squared error (SSE)**: $$\text{SSE}=\sum_{j=1}^K \sum_{i \in \text{OwnedBy}(c_j)}(e_i-c_j)^2=\sum_{j=1}^K\text{SSE}_j$$

* A cluster $j$ with an high $\text{SSE}_j$ has a low quality;
* If $\text{SSE}_j=0$ iff points are coincident with the centroid;
* $\text{SSE}$ decreases for increasing $K$, is zero when $K=N$.

In general minimizing $\text{SSE}$ is not a viable solution to choose the best $K$.

### Empty clusters
It may happen that at some step a centroid doesn't own any point.
This would changes the initial requirement of $K$ clusters, so choose a new centroid:
* Choose a point far away from the empty centroid;
* Or choose as new centroid a random point in the cluster with the maximum $\text{SSE}$ (worst cluster). In this way the cluster with the lowest quality will be split in two.

### Outliers
Outliers are point with high distance from their centroid, they give an high contribution to the $\text{SSE}$, and so the could have a bad influence on the clustering scheme (sometimes is a good idea to remove them).

### Common uses of $K$-means
* Can be easily used in the beginnign for data exploration (it's very fast);
* In a one-dimensional space it's a good way to discretize the values of a domain in non-uniform buckets;
* It's the basis for vector quantization, a technique for signal processing and compression or used for choosing the color palettes (color quantization).

### Complexity
The time complexity is $\mathcal{O}(TKND)$, with $T$ iterations, $K$ clusters and $N$ datapoints with $D$ dimensions.

## Evaluation of a clustering scheme
*Related to result, indipendent from clustering technique.*

For example clustering is an unsupervised method, no has ground truth to compare the result (basically no apriori information), so we need indexes to measure various properties of the clusters and the clustering scheme as a whole.
If some supervised data are available they can be used to evaluate the clustering scheme.

In $2$-dimensional spaces, clusters can be examined visually; in higher order spaces projections can help but formal methods are better.

The general objective is to distinguish patterns from random apparent regularities and find the best number of clusters.

The main measurement criteria are:
* **Cohesion** (to maximize): proximity of objects in the same cluster should be high;
* **Separation** (to minimize): distance between the nearest objects in the two clusters, distance between the most distance objects in the two clusters, distance between the centroids, etc... 



### Cohesion and separation
**Cohesion** - it's the sum of proximities between the elements of the cluster and the geometric center (prototype):
* **Centroid**: a point in the space whose coordinates are the means of the dataset;
* **Medoid**: an element of the dataset whose average dissimilarity with all the elements of the cluster is minimal. Used in contexts where means is not defined.
$$\text{Coh}(k_i)=\sum_{x \in k_i}\text{Prox}(x,c_i)$$

**Separation** - proximity between the prototypes (centroid or medoids):
$$\text{Sep}(k_i,k_j)=\text{Prox}(c_i,c_j)$$

![](https://i.ibb.co/47JM8rv/aegvevr.png)

**Global separation of a clustering scheme**: bigger clusters will have a bigger influence. The sum of squares between clusters is defined as:
$$\text{SSB}=\sum_{i=1}^K N_i \text{Dist}(c_i,c)^2$$

Let be $\text{TSS}$ the total sum of squares (sum of squared distances of the point from the global centroid), then $\text{TSS}=\text{SSE}+\text{SSB}$.
Since the sum is fixed then if we increase one the other decreases.

Note: $\text{TSS}$ is a global property of the dataset, independent from the clustering scheme.

### Evaluation
Each cluster can have its own evaluation: worst clusters can be considered for additional split and weakly separated pairs of clusters can be considere for merging.

Single objects can give negative contribution to the cohesion of a
cluster or to the separation between two clusters (border objects).


### [Silhouette](https://en.wikipedia.org/wiki/Silhouette_(clustering)) index of a cluster
Computation:
* For the $i$-th object inside the cluster, let be the the mean distance between $i$ and all other data points in the same cluster. Let's call it $a_i$.We can interpret $a(i)$ as a measure of how well $i$ is assigned to its cluster (the smaller the value, the better the assignment).
* For the $i$-th object and for each cluster other than its own, compute the mean distance of $i$ to all points in any other cluster, of which $i$ is not a member. Let's call it $b_i$.

For the $i$-th object the silhouette index is:
$$s_i=\frac{b_i-a_i}{\max(a_i,b_i)}\in [-1,1]$$

For the global index of a *cluster or clustering scheme* compute the average index over the *cluster or dataset*.

When the index is less than zero for an object, it means that there is a dominance of objects in other clusters at a distance smaller than the objects of the same cluster (well inside the cluster).

### Looking for the best number of clusters
![](https://i.ibb.co/j8hgnTg/photo-2021-01-03-11-34-36.jpg)

### Supervised measures
* Let's be available a labelled partition $P=\{P_1, \dots, P_L\}$ called **gold standard**;
* Consider the clustering scheme $K=\{k_1,\dots,k_K\}$

We want to compare them to validate the clustering techinque.
We have two main ways:
* **Classification oriented**: measure how the classes are distribuited among the clusters (confusion matrix, precision, recall, F-measure, etc...);
* **Similarity oriented**: analogous to binary data (rand index, Jaccard coefficient, etc...).

## Hierarchical clustering
It generates a nested structure of clusters that can be:
* **Agglomerative**: as a starting state each datapoint is a cluster. In each step the two less separated clusters are merged into one. 
A measure of separation between cluster is needed.
* **Divisive**: as a starting state the entire dataset is the only cluster.
In each step the cluster with the lowest cohesion is split.
A measure of cluster cohesion and a split procedure are needed.

The distance between sets is based on the distances between objects belonging to the two sets.

### Graph based separation between clusters
![](https://i.ibb.co/Dkfcx3g/photo-2021-01-03-11-50-58.jpg)

### Prototype based separation between clusters
There are several alternatives:
* Distance between the centroids;
* **Ward's method** (the most intuitive): given two sets with the respective $\text{SSE}$, the separation between the two is measured as the difference between the total $\text{SSE}$ resulting in case of merge and the sum of the original $\text{SSE}$.
Smaller separation implies a lower increase in the $\text{SSE}$ after merging (so if the difference doesn't increase a lot it wasn't very separated).

Note: when we merge the $\text{SSE}$ increases.

### Single linkage algorithm
1. Initialize the clusters, one for each objects;
2. Compute the distance matrix between the clusters (squared, symmetric, null main diagonal and size with the number of objects $N$);
3. While the number of clusters is greater than $1$:
> * Find the two clusters with lowest separation, say $k_r$ and $k_s$;
  * Merge them in a cluster;
  * Delete from the distance matrix row $r$ and column $s$ and insert a new row and column with the distances of the new cluster from the others: $$\text{dist}\bigl(k_k, k_{(r+s)}\bigr) = \min \Bigl( \text{dist}\bigl(k_k, k_r), \text{dist}\bigl(k_k, k_s\bigr) \Bigr)$$
  $$\forall k \in [1,K]$$

### Time and space complexity
![](https://i.ibb.co/DkmwvSh/comple.png)

### Generating the clustering scheme
The desidered clustering scheme is obtained by cutting the obtained dendogram at asome level: the choice is application-dependent.
![](https://i.ibb.co/cxPHY7C/photo-2021-01-03-12-10-22.jpg)
The horizontal axis is the **total dissimilarity** inside the clusters, which (obviously) increases for decreasing number of clusters.

The diameter of a cluster is the distance among the most separated objects:
* **Single linkage** tends to generate large clusters also at low level;
* **Complete linkage** tends to generate more compact clusters.

## Density based clustering
Clusters are high-density regions separated by low-density regions (not circular in general).
There are two main way to compute density:
* **Grid-based**: split the hyperspace into a regularly spaced grid and count the number of objects inside each grid element (hyperparameter: size of the grid);
* **Object-centered**: define the radius of an hypersphere and attach to each object the number of object which are inside that sphere (hyperparameter: radius).

### DBSCAN - Density based spatial clustering application with noise
**Intuition**: $p$ is a border point, $q$ is a core point.
![](https://i.ibb.co/yXyxd2f/eragvaer.png)

**Neighbourhood**: define a radius $\varepsilon$ and define a neighbourhood of a point the $\varepsilon$-hyperspere centered at that point. Points $p$ and $q$ are one in the neighbourhood of the other (the neighbourhood is symmetric).
![](https://i.ibb.co/nD59qV5/wscfasv.png)

**Direct density reachability**: define a threshold $\text{minPoints}$ and define as a **core** a point with at least $\text{minPoints}$ points in its neighbourhood, or as a **border** otherwise.

A point $p$ is **directly density reachable** from point $q$ iff: 
* $q$ is a core;
* $q$ is in the neighbourhood of $p$.

This property is not symmetric.

![](https://i.ibb.co/0svjgvS/one.png)

A point $p$ is **density reachable** from point $q$ iff: 
* $q$ is a core;
* There is a sequence of points $q_i$ such that $q_{i+1}$ is directly density reachable from $q_i \forall i \in [1,nq]$;
* $q_1$ is directly reachable from $q$; 
* $p$ is directly reachable from $q_{nq}$. 

This property is not symmetric.

![](https://i.ibb.co/NsvG8PK/two.png)

A point $p$ is **density connected** to point $q$ iff:
* There is a point $s$ such that $p$ and $q$ are density reachable from $s$.

This property is symmetric.

![](https://ibb.co/THvZzvH)

**Generation of clusters**: a cluster is a maximal set of points connected by *density*. Border point which are not connected by density to any core point are labelled as noise.

This method:
* Finds clusters of any shape;
* Is robust to noise;
* There are problem if the densities of clusters vary widely (it's distance-based $\mathcal{O}(N^2)$).

### KDE - Kernel density estimation
A technique developed in statistics and pattern mining. 
It's not based on distances but in neighbourhoods but on function distributions that describe the data.

The overall density function is the sum of the **influence or kernel functions** associated with each point.
The kernel function must be symmetric and monotonically decreasing and usually has a parameter to set the decreasing rate.

### DENCLUE algorithm - Density clustering algorithm
1. Derive a density function for the space occupied by the data points:
1. Identify the point that are local maxima;
1. Associate each point with a **density attractor** by moving in the directions of maximum increase in density;
1. Define clusters consisting of points associated with a particular density attractor;
1. Discard clusters whose density attractor has a density less than a user-specified threshold $\xi$;
1. Combine clusters that are connected by a path of points that all have a density of $\xi$ or higher.

The algorithm has a strong theoretical foundation (DBSCAN is a special case of DENCLUE where the density attractor is a step function).
It's good at dealing with the noise and cluster with different shapes and size but have some troubles with higher dimensional data and clusters with different densities.
Expensive computation $\mathcal{O}(N^2)$.


## Model based clustering
Estimate the parameters of a statistical model to maximize the ability of the model to explain the data.
The main techinque is to use the mixture models: view the data as a set of observation from a mixture of different probability distributions.
Usually the base model is a multivariate normal.

The estimation is usually done using the maximum likelihood: given a set of data $\varepsilon$, the probability of the data is called likelihood function.

Hypotesis: attributes are independent.

### EM - Expectation maximization
If the data can be approximated by a single distribution the derivation of the parameters is straightforward.
In the general case, with many mixed distributions, the EM algorithm is used.
![](https://i.ibb.co/h9ttZz5/algo.jpg)

## Recap
* **Partitioning**: iteratively find partitions in the dataset optimizing some quality criterion (i.e. $k$-means);
* **Hierarchic**: recursively compute a structured hierarchy of subsets;
* **Density based**: compute metrics and aggregates clusters in high density areas;
* **Model based**: assume a model for the distribution and find the model parameters that guarantee the best fitting to the data.

Effectiveness decreases with dimensionality $D$ and noise level.

Computational cost increases with dataset size $N$ and dimensionality $D$.